---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
def format_df(df, mask):
    """
    This function adjust dates, calculate the number of days between the date 
    the ticket is issued and the date of the hearing, and fill null values in a dataframe 
    """
    
    # Import relevant libraries
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import datetime as dt
    
    # Create a copy of the dataframe to avoid SettinWithCopyWarnings
    df = df.copy()
    
    # Call and adjust the mask according to the dataframe we are going to use
    try:
        # For our train dataset
        df = df[mask]
    except:
        # For our test dataset
        df = df[mask[:-1]]
    
    # Adjust time features and create a date_diff variable
    df['hearing_date'] = pd.to_datetime(df['hearing_date'], errors = 'coerce')
    df['hearing_date'] = df['hearing_date'].fillna(method = 'pad')
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'], errors = 'coerce')
    df['ticket_issued_date'] = df['ticket_issued_date'].fillna(method = 'pad')
    df['date_diff'] = (df['hearing_date'] - df['ticket_issued_date'])
    df['date_diff'] = df['date_diff'].dt.days
    
    # Fill nulls in the rest of the features in our dataframe
    for column in df.columns:
        df[column] = df[column].fillna(method = 'pad')
    
    # Return dataframe
    return df

def assign_categories(df, mask, remove):
    """
    This function convert selected features into categories.  Converting features into categories
    reduce the processing time into our ML model
    """
    # Import relevant libraries
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import datetime as dt
    
    # Create a copy of the dataframe to avoid SettinWithCopyWarnings
    df = df.copy()
    
    # Define mask for categories
    category_mask = [i for i in mask if i not in remove]
    
    # Convert features into categories
    try:
        # For our train dataset
        for col in category_mask:
            df[col] = df[col].astype('category')
        #df[category_mask] = df[category_mask].astype('category') # This works in Python 3.5.4
    except:
        # For our test dataset
        for col in category_mask[:-1]:
            df[col] = df[col].astype('category')
        #df[category_mask[:-1]] = df[category_mask[:-1]].astype('category') # This works in Python 3.5.4
    
    # Return dataframe
    return df

In [2]:
def blight_model():
    """
    Last update: 03/10/2019
    Author: John Hill Escobar
    
    This function intends to build up a series with the probabilities that 
    the corresponding blight ticket will be paid on time.

    The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).
    
    This is the last evaluation of the course Applied Machine Learning with Python
    """
    
    # Import relevant libraries
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import datetime as dt
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score, roc_curve, auc
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    
    # Read dataframes
    # I included the argument dtype on the training dataset to avoid a DtypeWarning on columns (11, 12, 31)
    train = pd.read_csv('readonly/train.csv', encoding='cp1252', 
                           dtype = {'zip_code': object, 'non_us_str_code': object, 'grafitti_status': object})
    test = pd.read_csv('readonly/test.csv', encoding='cp1252')
    address = pd.read_csv('readonly/addresses.csv')
    latlong = pd.read_csv('readonly/latlons.csv')
    
    # Join address and latlong in one dataframe.  Then, join address to both train and test data
    address = address.set_index('address').join(latlong.set_index('address'), how = 'left')
    train = train.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
    test = test.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
    
    # Drop NaN on target value 'compliance' --> we are just looking for compliants and or not compliants
    # Although the target variable 'compliance' is dtype('float64'). So, I will adjust it
    # to meake it categorical
    train = train.dropna(subset = ['compliance'])
    train['compliance'] = train['compliance'].astype(int)
    
    # I created a spreadsheet in which I stored the features I wanted to use on the training and
    # and testing dataframes.  Then, I created the mask from there.
    #mask = pd.read_excel('model_columns_final.xlsx')
    #mask = mask['columnas']
    #mask = mask.tolist()
    
    # I commented out this part of the code as the autograder will get an error from it.
    # Instead, I just created a variable called "mask", which I used to keep all the attributions 
    # I will use on this excercise.
    
    # Create the mask variables with all the variables you want to use
    mask = ['hearing_date', 
            'ticket_issued_date', 
            'discount_amount',
            'disposition', 
            'fine_amount', 
            'judgment_amount', 
            'late_fee', 
            'state', 
            'violation_code', 
            'lat',
            'lon',
            'compliance']
    
    # Create the final training dataframe and adjust its data types as well as create a new variable 
    # named "date_diff"
    train = format_df(train, mask)
    
    # Create the final test dataframe and adjust its data types as well as create a new variable 
    # named "date_diff"
    test = format_df(test, mask)
    
    # Create features to remove from our mask to change the features to category, when applicable
    # Create the mask variables with all the variables you want to use
    remove = {'hearing_date',
              'ticket_issued_date',
              'lat',
              'lon'}
    
    # Convert applicable features into categories
    train  = assign_categories(train, mask, remove)
    test  = assign_categories(test, mask, remove)
    
    
    # Define y_train, X_train, and X_test
    y_train = train['compliance'].copy()
    X_train = train.drop('compliance', axis = 1).copy()
    X_test = test.copy()
    
    # Label encode your categorical features
    label_encoder = LabelEncoder()

    for col in X_train.select_dtypes(include=['category']).columns:
        label_encoder.fit(X_train[col].append(X_test[col], ignore_index = True))
        X_train[col] = label_encoder.transform(X_train[col])
        X_test[col] = label_encoder.transform(X_test[col])
    
    # At this poing, we will get an error when running our model we keep our dates.   
    # Therefore, I am dropping them
    X_train = X_train.drop(['hearing_date', 'ticket_issued_date'], axis = 1)
    X_test = X_test.drop(['hearing_date', 'ticket_issued_date'], axis = 1)
    
    # Finally, let's run our random forest model
    X_trainer, X_tester, y_trainer, y_tester = train_test_split(X_train, y_train)
    
    regr_rf = RandomForestRegressor()
    grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
    grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
    grid_clf_auc.fit(X_trainer, y_trainer)
    print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    answer = pd.Series(grid_clf_auc.predict(X_test),index=X_test.index)
    
        
    # Return probalities series
    return answer

blight_model()

Grid best parameter (max. AUC):  {'n_estimators': 100, 'max_depth': 30}
Grid best score (AUC):  0.7985928598772787


ticket_id
284932    0.130000
285362    0.043042
285361    0.122727
285338    0.000000
285346    0.180960
285345    0.010000
285347    0.040254
285342    0.940000
285530    0.000000
284989    0.000000
285344    0.001392
285343    0.000054
285340    0.000054
285341    0.052078
285349    0.270000
285348    0.040000
284991    0.010370
285532    0.000000
285406    0.001847
285001    0.021471
285006    0.030054
285405    0.000000
285337    0.020000
285496    0.018324
285497    0.041523
285378    0.000000
285589    0.020208
285585    0.070797
285501    0.084099
285581    0.001155
            ...   
376367    0.011375
376366    0.023464
376362    0.036821
376363    0.086821
376365    0.011375
376364    0.023464
376228    0.052077
376265    0.233708
376286    0.050000
376320    0.056381
376314    0.031403
376327    0.281608
376385    0.287262
376435    0.870000
376370    0.960000
376434    0.003650
376459    0.120956
376478    0.000000
376473    0.026292
376484    0.020000
376482    0.100000
37

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
train = pd.read_csv('readonly/train.csv', encoding='cp1252', 
                           dtype = {'zip_code': object, 'non_us_str_code': object, 'grafitti_status': object})
test = pd.read_csv('readonly/test.csv', encoding='cp1252')
address = pd.read_csv('readonly/addresses.csv')
latlong = pd.read_csv('readonly/latlons.csv')

In [5]:
train.shape

(250306, 34)

In [6]:
test.shape

(61001, 27)

In [7]:
# Drop NaN on target value 'compliance' --> we are just looking for compliants and or not compliants
# Although the target variable 'compliance' is dtype('float64'). So, I will adjust it
# to meake it categorical
train = train.dropna(subset = ['compliance'])
train['compliance'] = train['compliance'].astype(int)

In [8]:
# Join address and latlong in one dataframe.  Then, join address to both train and test data
address = address.set_index('address').join(latlong.set_index('address'), how = 'left')
train = train.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
test = test.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')

In [9]:
train.shape

(159880, 35)

In [10]:
test.shape

(61001, 28)

In [11]:
# Create the mask variables with all the variables you want to use
mask = ['hearing_date', 
        'ticket_issued_date', 
        'discount_amount',
        'disposition', 
        'fine_amount', 
        'judgment_amount', 
        'late_fee', 
        'state', 
        'violation_code', 
        'lat',
        'lon',
        'compliance']

In [12]:
train = train[mask] #.dropna()
train['hearing_date'] = pd.to_datetime(train['hearing_date'], errors = 'coerce')
train['hearing_date'] = train['hearing_date'].fillna(method = 'pad')
train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'], errors = 'coerce')
train['ticket_issued_date'] = train['ticket_issued_date'].fillna(method = 'pad')
train['date_diff'] = (train['hearing_date'] - train['ticket_issued_date'])
train['date_diff'] = train['date_diff'].dt.days
#train = train[~train['date_diff'] < 0]

In [13]:
train.shape

(159880, 13)

In [14]:
def test_df(df, mask):
    try:
        return test[mask]
    except:
        return test[mask[:-2]]
    
#test = test_df(test, mask)
#test

In [15]:
def format_df(df, mask):
    df = df.copy()
    try:
        df = df[mask] #.dropna()
    except:
        df = df[mask[:-1]]
        
    df['hearing_date'] = pd.to_datetime(df['hearing_date'], errors = 'coerce')
    df['hearing_date'] = df['hearing_date'].fillna(method = 'pad')
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'], errors = 'coerce')
    df['ticket_issued_date'] = df['ticket_issued_date'].fillna(method = 'pad')
    df['date_diff'] = (df['hearing_date'] - df['ticket_issued_date'])
    df['date_diff'] = df['date_diff'].dt.days
    for column in df.columns:
        df[column] = df[column].fillna(method = 'pad')
    
    return df

train = format_df(train, mask)
train.head()

,hearing_date,ticket_issued_date,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,compliance,date_diff
ticket_id,,,,,,,,,,,,,
22056,2005-03-21 10:30:00,2004-03-16 11:40:00,0.0,Responsible by Default,250.0,305.0,25.0,IL,9-1-36(a),42.390729,-83.124268,0,369
27586,2005-05-06 13:30:00,2004-04-23 12:30:00,0.0,Responsible by Determination,750.0,855.0,75.0,MI,61-63.0600,42.326937,-83.135118,1,378
22046,2005-03-21 10:30:00,2004-05-01 11:50:00,0.0,Responsible by Default,250.0,305.0,25.0,CA,9-1-36(a),42.145257,-83.208233,0,323
18738,2005-02-22 15:00:00,2004-06-14 14:15:00,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0500,42.433466,-83.023493,0,253
18735,2005-02-22 15:00:00,2004-06-16 12:30:00,0.0,Responsible by Default,100.0,140.0,10.0,MI,61-63.0100,42.388641,-83.037858,0,251


In [16]:
train.shape

(159880, 13)

In [17]:
test = format_df(test, mask)
test.head()

,hearing_date,ticket_issued_date,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,date_diff
ticket_id,,,,,,,,,,,,
284932,2012-01-19 09:00:00,2012-01-04 14:00:00,0.0,Responsible by Default,200.0,250.0,20.0,MI,22-2-61,42.407581,-82.986642,14
285362,2012-02-06 09:00:00,2012-01-05 09:50:00,0.0,Responsible by Default,1000.0,1130.0,100.0,MI,22-2-88(b),42.426239,-83.238259,31
285361,2012-02-06 09:00:00,2012-01-05 09:50:00,0.0,Responsible by Default,100.0,140.0,10.0,MI,22-2-43,42.426239,-83.238259,31
285338,2012-02-07 09:00:00,2012-01-05 10:25:00,0.0,Responsible by Default,200.0,250.0,20.0,MI,22-2-88(b),42.309661,-83.122426,32
285346,2012-02-14 09:00:00,2012-01-05 10:20:00,0.0,Responsible by Default,100.0,140.0,10.0,MI,22-2-45,42.308830,-83.121116,39


In [18]:
test.shape

(61001, 12)

In [19]:
for column in test.columns:
    test[column] = test[column].fillna(method = 'pad')
    


In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 12 columns):
hearing_date          61001 non-null datetime64[ns]
ticket_issued_date    61001 non-null datetime64[ns]
discount_amount       61001 non-null float64
disposition           61001 non-null object
fine_amount           61001 non-null float64
judgment_amount       61001 non-null float64
late_fee              61001 non-null float64
state                 61001 non-null object
violation_code        61001 non-null object
lat                   61001 non-null float64
lon                   61001 non-null float64
date_diff             61001 non-null int64
dtypes: datetime64[ns](2), float64(6), int64(1), object(3)
memory usage: 6.1+ MB


In [21]:
#test['state'].fillna(method = 'pad')

In [22]:
# test[state].info()

In [23]:
for column in train.columns:
    train[column] = train[column].fillna(method = 'pad')

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 284333
Data columns (total 13 columns):
hearing_date          159880 non-null datetime64[ns]
ticket_issued_date    159880 non-null datetime64[ns]
discount_amount       159880 non-null float64
disposition           159880 non-null object
fine_amount           159880 non-null float64
judgment_amount       159880 non-null float64
late_fee              159880 non-null float64
state                 159880 non-null object
violation_code        159880 non-null object
lat                   159880 non-null float64
lon                   159880 non-null float64
compliance            159880 non-null int32
date_diff             159880 non-null int64
dtypes: datetime64[ns](2), float64(6), int32(1), int64(1), object(3)
memory usage: 16.5+ MB


In [25]:
#train['violation_code'].value_counts()

In [26]:
# Create features to remove from our mask
# Create the mask variables with all the variables you want to use
remove = {'hearing_date', 
        'ticket_issued_date',
        'lat',
        'lon'}

category_mask = [i for i in mask if i not in remove]
category_mask

['discount_amount',
 'disposition',
 'fine_amount',
 'judgment_amount',
 'late_fee',
 'state',
 'violation_code',
 'compliance']

In [27]:
train[category_mask] = train[category_mask].astype('category')

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 284333
Data columns (total 13 columns):
hearing_date          159880 non-null datetime64[ns]
ticket_issued_date    159880 non-null datetime64[ns]
discount_amount       159880 non-null category
disposition           159880 non-null category
fine_amount           159880 non-null category
judgment_amount       159880 non-null category
late_fee              159880 non-null category
state                 159880 non-null category
violation_code        159880 non-null category
lat                   159880 non-null float64
lon                   159880 non-null float64
compliance            159880 non-null category
date_diff             159880 non-null int64
dtypes: category(8), datetime64[ns](2), float64(2), int64(1)
memory usage: 8.7 MB


In [29]:
test[category_mask[:-1]] = test[category_mask[:-1]].astype('category')

In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 12 columns):
hearing_date          61001 non-null datetime64[ns]
ticket_issued_date    61001 non-null datetime64[ns]
discount_amount       61001 non-null category
disposition           61001 non-null category
fine_amount           61001 non-null category
judgment_amount       61001 non-null category
late_fee              61001 non-null category
state                 61001 non-null category
violation_code        61001 non-null category
lat                   61001 non-null float64
lon                   61001 non-null float64
date_diff             61001 non-null int64
dtypes: category(7), datetime64[ns](2), float64(2), int64(1)
memory usage: 3.4 MB


In [31]:
def assign_categories(df, mask, remove):
    """
    
    """
    
    # Create dataframe copy
    df = df.copy()
    
    # Define mask for categories
    category_mask = [i for i in mask if i not in remove]
    
    # Convert features into categories
    try:
        df[category_mask] = df[category_mask].astype('category')
    except:
        df[category_mask[:-1]] = df[category_mask[:-1]].astype('category')
    
    # Return dataframe
    return df

In [32]:
train  = assign_categories(train, mask, remove)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 284333
Data columns (total 13 columns):
hearing_date          159880 non-null datetime64[ns]
ticket_issued_date    159880 non-null datetime64[ns]
discount_amount       159880 non-null category
disposition           159880 non-null category
fine_amount           159880 non-null category
judgment_amount       159880 non-null category
late_fee              159880 non-null category
state                 159880 non-null category
violation_code        159880 non-null category
lat                   159880 non-null float64
lon                   159880 non-null float64
compliance            159880 non-null category
date_diff             159880 non-null int64
dtypes: category(8), datetime64[ns](2), float64(2), int64(1)
memory usage: 8.7 MB


In [33]:
test  = assign_categories(test, mask, remove)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 12 columns):
hearing_date          61001 non-null datetime64[ns]
ticket_issued_date    61001 non-null datetime64[ns]
discount_amount       61001 non-null category
disposition           61001 non-null category
fine_amount           61001 non-null category
judgment_amount       61001 non-null category
late_fee              61001 non-null category
state                 61001 non-null category
violation_code        61001 non-null category
lat                   61001 non-null float64
lon                   61001 non-null float64
date_diff             61001 non-null int64
dtypes: category(7), datetime64[ns](2), float64(2), int64(1)
memory usage: 3.4 MB


In [34]:
y_train = train['compliance'].copy()

In [35]:
y_train.value_counts()

0    148283
1     11597
Name: compliance, dtype: int64

In [36]:
y_train.shape

(159880,)

In [37]:
X_train = train.drop('compliance', axis = 1).copy()

In [38]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 284333
Data columns (total 12 columns):
hearing_date          159880 non-null datetime64[ns]
ticket_issued_date    159880 non-null datetime64[ns]
discount_amount       159880 non-null category
disposition           159880 non-null category
fine_amount           159880 non-null category
judgment_amount       159880 non-null category
late_fee              159880 non-null category
state                 159880 non-null category
violation_code        159880 non-null category
lat                   159880 non-null float64
lon                   159880 non-null float64
date_diff             159880 non-null int64
dtypes: category(7), datetime64[ns](2), float64(2), int64(1)
memory usage: 8.6 MB


In [39]:
X_test = test.copy()

In [40]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 12 columns):
hearing_date          61001 non-null datetime64[ns]
ticket_issued_date    61001 non-null datetime64[ns]
discount_amount       61001 non-null category
disposition           61001 non-null category
fine_amount           61001 non-null category
judgment_amount       61001 non-null category
late_fee              61001 non-null category
state                 61001 non-null category
violation_code        61001 non-null category
lat                   61001 non-null float64
lon                   61001 non-null float64
date_diff             61001 non-null int64
dtypes: category(7), datetime64[ns](2), float64(2), int64(1)
memory usage: 3.4 MB


In [41]:
df_dtypes = np.array(X_train.dtypes)
df_numericDtypes= [x.kind in 'bifc' for x in df_dtypes]
df_numericDtypes

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True]

In [42]:
X_train.select_dtypes(include=['category']).columns

Index(['discount_amount', 'disposition', 'fine_amount', 'judgment_amount',
       'late_fee', 'state', 'violation_code'],
      dtype='object')

In [43]:
X_train.dtypes[X_train.dtypes != 'int64'][X_train.dtypes != 'float64'][X_train.dtypes != 'datetime64[ns]']  

discount_amount    category
disposition        category
fine_amount        category
judgment_amount    category
late_fee           category
state              category
violation_code     category
dtype: object

In [44]:
#label_encoder = LabelEncoder()
#label_encoder.fit(X_train['discount_amount'].append(X_test['discount_amount'], ignore_index=True))
#X_train['discount_amount'] = label_encoder.transform(X_train['discount_amount'])
#X_test['discount_amount'] = label_encoder.transform(X_test['discount_amount'])

In [45]:
#X_test['discount_amount'].dtype

In [46]:
label_encoder = LabelEncoder()

for col in X_train.select_dtypes(include=['category']).columns:
    label_encoder.fit(X_train[col].append(X_test[col], ignore_index = True))
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])
    
    



In [47]:
X_train = X_train.drop(['hearing_date', 'ticket_issued_date'], axis = 1)
X_train.head()

,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,date_diff
ticket_id,,,,,,,,,,
22056,0,5,20,51,17,16,175,42.390729,-83.124268,369
27586,0,6,42,222,35,24,98,42.326937,-83.135118,378
22046,0,5,20,51,17,6,175,42.145257,-83.208233,323
18738,0,5,42,222,35,24,97,42.433466,-83.023493,253
18735,0,5,11,18,9,24,96,42.388641,-83.037858,251


In [48]:
X_test = X_test.drop(['hearing_date', 'ticket_issued_date'], axis = 1)
X_test.head()

,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,date_diff
ticket_id,,,,,,,,,,
284932,0,5,17,39,15,24,38,42.407581,-82.986642,14
285362,0,5,45,273,38,24,57,42.426239,-83.238259,31
285361,0,5,11,18,9,24,25,42.426239,-83.238259,31
285338,0,5,17,39,15,24,57,42.309661,-83.122426,32
285346,0,5,11,18,9,24,27,42.308830,-83.121116,39


In [49]:
 X_trainer, X_tester, y_trainer, y_tester = train_test_split(X_train, y_train)

In [50]:
"""
regr_rf = RandomForestRegressor()
grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
grid_clf_auc = GridSearchCV(regr_, y_train)
print('Grid best parametrf, param_grid=grid_values, scoring='roc_auc')
grid_clf_auc.fit(X_train (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

pd.Series(grid_clf_auc.predict(X_test), index=X_test.index)
"""

"\nregr_rf = RandomForestRegressor()\ngrid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}\ngrid_clf_auc = GridSearchCV(regr_, y_train)\nprint('Grid best parametrf, param_grid=grid_values, scoring='roc_auc')\ngrid_clf_auc.fit(X_train (max. AUC): ', grid_clf_auc.best_params_)\nprint('Grid best score (AUC): ', grid_clf_auc.best_score_)\n\npd.Series(grid_clf_auc.predict(X_test), index=X_test.index)\n"

In [51]:
"""
regr_rf = RandomForestRegressor()
grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
grid_clf_auc.fit(X_trainer, y_trainer)
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    
pd.Series(grid_clf_auc.predict(X_test),index=X_test.index)
"""

"\nregr_rf = RandomForestRegressor()\ngrid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}\ngrid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')\ngrid_clf_auc.fit(X_trainer, y_trainer)\nprint('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)\nprint('Grid best score (AUC): ', grid_clf_auc.best_score_)\n    \npd.Series(grid_clf_auc.predict(X_test),index=X_test.index)\n"